In [ ]:
import time
import os
import json

import pandas as pd
import dataset
import haversine
from keplergl import KeplerGl
from simplification.cutil import simplify_coords_idx

In [ ]:
%%time
# TODO move to helper module

BASE_POINT = (44.01016, 56.31832)
DISTANCE_LIMIT_KM = 600
SIMPLIFY_THRESHOLD = 0.0001
LIMIT_ACTIVITIES = None
CONFIG_FILE_LOCATION = 'map_config.json'
DATE_GTE = '2020-07-24'

NO_LIMITS = True
if NO_LIMITS:
    DATE_GTE = None
    LIMIT_ACTIVITIES = None
    DISTANCE_LIMIT_KM = 0
    
filters = {
    '_limit': LIMIT_ACTIVITIES,
}
if DATE_GTE:
    filters['datetime'] = {'gte': DATE_GTE}


def fix_order(entry):
    entry['latlng'] = [(x[1], x[0]) for x in entry['latlng']]
    return entry


db = dataset.connect('sqlite:///stravaweb/strava_activities.sqlite')
df = pd.DataFrame(fix_order(x) for x in db['activities'].all(**filters) if x['latlng'])
df['datetime'] = pd.to_datetime(df['datetime'])
df['datetime'] = df['datetime'] + pd.Timedelta(hours=3)  # hack to make Moscow time


def gen_features(df, simplify_threshold=0.001):
    for latlng, times, base_datetime in zip(df.latlng, df.time, df.datetime):
        base_time = base_datetime.timestamp()
        indexes = simplify_coords_idx(latlng, simplify_threshold)
        #          lat           lon           alt   time                
        coords = [[latlng[i][0], latlng[i][1], 0, int(base_time + times[i])] for i in indexes]
        yield {
            "type": "FeatureCollection",
            "features": [
              {
                "type": "Feature",
                "geometry": {
                  "type": "LineString",
                  "coordinates": coords,
                }
              }
            ]
          }

df['track'] = list(gen_features(df, SIMPLIFY_THRESHOLD))
df['athlete_url'] = df.apply(
    lambda x: 'https://www.strava.com/athletes/{}'.format(x['athlete_id']),
    axis=1
)
df['activity_url'] = df.apply(
    lambda x: 'https://www.strava.com/activities/{}'.format(x['activity_id']),
    axis=1
)
df['distance_to_base_km'] = df.apply(lambda x: haversine.haversine(x['latlng'][0], BASE_POINT), axis=1)
if DISTANCE_LIMIT_KM:
    df = df[df.distance_to_base_km < DISTANCE_LIMIT_KM]

df = df.drop(columns=['velocity_smooth', 'id', 'latlng', 'time'])
df['datetime'] = df['datetime'].astype(str)  # Need to convert datetime back to string, because kepler can't convert it to JSON

print('Selected activities', df.shape[0])
df.head()

In [ ]:
config = None
if os.path.exists(CONFIG_FILE_LOCATION):
    with open(CONFIG_FILE_LOCATION) as fp:
        config = json.load(fp)
        
my_map = KeplerGl(height=800, config=config)
my_map.add_data(df, 'tracks')
my_map

In [ ]:
my_map.save_to_html(file_name='tracks_map.html')
# https://github.com/keplergl/kepler.gl/issues/1162 
# modify site-packages/keplergl/static/keplergl.html to save tooltips config

In [ ]:
# Uncomment to overwrite map config after tuning

def save_config(m):
    with open(CONFIG_FILE_LOCATION, 'w') as fp:
        json.dump(m.config, fp)
# save_config(my_map)